---
<img src="CoronavirusImage.png" align="center"/>

# Group: CovidCats

### *Members:  Aldo, Andrew, Araz, Jane & Veohnti*

## Project:   Extract – Transform – Load

---

In the midst of the 2020 global coronavirus pandemic, this project ETLs the COVID-19 daily case data from World Health Organization (WHO) reports and others, *as compiled by Johns Hopkins University (JHU) and promulgated on [JHU's GitHub repository](https://github.com/CSSEGISandData/COVID-19)*, in order to ultimately **compare growth trajectories by country on comparable timescales**, beginning from the day on which each respective country had one hundred confirmed cases of COVID-19 (*i.e.*, 100 or more cases = Day 0).

The daily time series case data is available on JHU's GitHub repository in three (3) separate CSV files by case type:  

+ confirmed cases, 
+ deaths, and 
+ recovered cases.

We used an API from [GitHub's Developer Tools](https://developer.github.com/v3/repos/contents/#get-contents "Click to visit GitHub's Developer documentation") to pull the most current daily data so that our extracted, cleaned dataframes and loaded tables could automatically update each time the ETL process was performed.

---

> ### Step 1 – Extract
>
> First we pulled the raw data from JHU's GitHub repository, reading JHU's CSV files into Pandas dataframes:

---

In [1]:
# Install PyGitHub for extracting data via GitHub API
get_ipython().system(' pip install PyGithub')

In [256]:
# Import dependencies (including those needed for cleaning later...) and a separately saved config.py file containing a GitHub personal access token (API key).
import pandas as pd
import os
import numpy as np
import datetime
from config import git_key

In [257]:
# Importing the Population file
pop_file = "WPP2019_POP_F01_1_TOTAL_POPULATION_BOTH_SEXES.xlsx"

df_pop = pd.read_excel(pop_file)

In [258]:
# View raw dataframe of World Populations (from the UN)
df_pop

,Index,Variant,"Region, subregion, country or area *",Notes,Country code,Type,Parent code,1950,1951,1952,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,1,Estimates,WORLD,NaN,900,World,0,2.53643e+06,2.58403e+06,2.63086e+06,...,7.04119e+06,7.12583e+06,7.21058e+06,7.29529e+06,7.3798e+06,7.46402e+06,7.54786e+06,7.63109e+06,7.71347e+06,7.7948e+06
1,2,Estimates,UN development groups,a,1803,Label/Separator,900,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,3,Estimates,More developed regions,b,901,Development Group,1803,814819,824004,833720,...,1.23956e+06,1.24411e+06,1.24845e+06,1.25262e+06,1.25662e+06,1.26048e+06,1.26415e+06,1.26756e+06,1.27063e+06,1.2733e+06
3,4,Estimates,Less developed regions,c,902,Development Group,1803,1.72161e+06,1.76003e+06,1.79714e+06,...,5.80164e+06,5.88171e+06,5.96213e+06,6.04268e+06,6.12317e+06,6.20354e+06,6.28371e+06,6.36353e+06,6.44284e+06,6.52149e+06
4,5,Estimates,Least developed countries,d,941,Development Group,902,195428,199180,203015,...,856471,876867,897793,919223,941131,963520,986385,1.00969e+06,1.03339e+06,1.05744e+06
5,6,Estimates,"Less developed regions, excluding least develo...",e,934,Development Group,902,1.52618e+06,1.56085e+06,1.59413e+06,...,4.94517e+06,5.00485e+06,5.06434e+06,5.12345e+06,5.18204e+06,5.24002e+06,5.29733e+06,5.35384e+06,5.40945e+06,5.46406e+06
6,7,Estimates,"Less developed regions, excluding China",NaN,948,Development Group,1803,1.15742e+06,1.17993e+06,1.20396e+06,...,4.39431e+06,4.46655e+06,4.53916e+06,4.612e+06,4.68498e+06,4.75802e+06,4.83109e+06,4.90415e+06,4.9772e+06,5.05021e+06
7,8,Estimates,Land-locked Developing Countries (LLDC),f,1636,Special other,1803,103803,105870,108079,...,430709,441057,451699,462624,473817,485277,496988,508906,520973,533143
8,9,Estimates,Small Island Developing States (SIDS),g,1637,Special other,1803,23771.1,24209.1,24684.6,...,66034.5,66778.7,67490.7,68180.4,68855.5,69515,70157.5,70791.8,71428.8,72076.1
9,10,Estimates,World Bank income groups,NaN,1802,Label/Separator,900,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [283]:
#Rename Columns to match across DFs
df_pop = df_pop.rename(columns={"Region, subregion, country or area *" : "Country/Region"})
df_pop

,Index,Variant,Country/Region,Notes,Country code,Type,Parent code,1950,1951,1952,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
26,27,Estimates,Burundi,NaN,108,Country/Area,910,2308.93,2360.44,2406.03,...,8958.41,9245.99,9540.3,9844.3,10160,10488,10827,11175.4,11530.6,11890.8
27,28,Estimates,Comoros,NaN,174,Country/Area,910,159.459,163.146,166.538,...,706.578,723.865,741.511,759.39,777.435,795.597,813.89,832.322,850.891,869.595
28,29,Estimates,Djibouti,NaN,262,Country/Area,910,62,63.313,64.744,...,853.671,868.136,883.296,898.707,913.998,929.117,944.1,958.923,973.557,988.002
29,30,Estimates,Eritrea,NaN,232,Country/Area,910,822.347,835,849.258,...,3213.97,3250.1,3281.45,3311.44,3342.82,3376.56,3412.89,3452.8,3497.12,3546.43
30,31,Estimates,Ethiopia,NaN,231,Country/Area,910,18128,18467,18819.7,...,90139.9,92727,95385.8,98094.3,100835,103603,106400,109224,112079,114964
31,32,Estimates,Kenya,NaN,404,Country/Area,910,6076.76,6242.11,6415.95,...,43178.3,44343.5,45520,46700.1,47878.3,49051.5,50221.1,51392.6,52574,53771.3
32,33,Estimates,Madagascar,NaN,450,Country/Area,910,4083.55,4168.39,4256.76,...,21744,22346.6,22961.3,23589.9,24234.1,24894.4,25570.5,26262.3,26969.3,27691
33,34,Estimates,Malawi,NaN,454,Country/Area,910,2953.87,3011.79,3072.39,...,14962.1,15396,15839.3,16289.5,16745.3,17205.3,17670.2,18143.2,18628.7,19130
34,35,Estimates,Mauritius,1,480,Country/Area,910,493.258,506.428,521.191,...,1251.07,1253.49,1255.46,1257.35,1259.46,1261.87,1264.5,1267.18,1269.67,1271.77
35,36,Estimates,Mayotte,2,175,Country/Area,910,15.138,15.732,16.444,...,214.937,221.147,227.376,233.658,240.011,246.46,252.962,259.526,266.153,272.813


In [260]:
# Define function to extract current coronavirus data from Johns Hopkins' Github repository and read CSV into a dataframe
def repo_to_df(git_key, branch):
    
    # Import dependencies
    from github import Github
    import requests
    import io
    
    # Create a Github API instance using an access key token
    g = Github(git_key)
    
    # Set Github repository name for GET requests to retrieve coronavirus data (CSV files)
    repo = g.get_repo("CSSEGISandData/COVID-19")
    contents = repo.get_contents(branch)
    
    # Decode and read CSV into dataframe 
    df = pd.read_csv(io.StringIO(contents.decoded_content.decode('utf-8')))
    return df

In [261]:
# Use defined function (above) to extract CSVs of coronavirus data from Github into dataframes (confirmed cases, deaths...)
confirmed_df = repo_to_df(git_key,"/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
deaths_df = repo_to_df(git_key,"/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
recovered_df = repo_to_df(git_key,"/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")

In [262]:
# View raw dataframe of confirmed cases
confirmed_df

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20
0,NaN,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,...,40,74,84,94,110,110,120,170,174,237
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,104,123,146,174,186,197,212,223,243,259
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,230,264,302,367,409,454,511,584,716,847
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,133,164,188,224,267,308,334,370,376,390
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,3,3,3,4,4,5,7,7,7,8
5,NaN,Antigua and Barbuda,17.060800,-61.796400,0,0,0,0,0,0,...,3,3,3,7,7,7,7,7,7,7
6,NaN,Argentina,-38.416100,-63.616700,0,0,0,0,0,0,...,301,387,387,502,589,690,745,820,1054,1054
7,NaN,Armenia,40.069100,45.038200,0,0,0,0,0,0,...,235,249,265,290,329,407,424,482,532,571
8,Australian Capital Territory,Australia,-35.473500,149.012400,0,0,0,0,0,0,...,32,39,39,53,62,71,77,78,80,84
9,New South Wales,Australia,-33.868800,151.209300,0,0,0,0,3,4,...,669,818,1029,1219,1405,1617,1791,2032,2032,2182


In [263]:
# View raw dataframe of deaths
deaths_df

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20
0,NaN,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,...,1,1,2,4,4,4,4,4,4,4
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,4,5,5,6,8,10,10,11,15,15
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,17,19,21,25,26,29,31,35,44,58
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,1,1,1,3,3,3,6,8,12,14
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,0,0,0,0,0,0,2,2,2,2
5,NaN,Antigua and Barbuda,17.060800,-61.796400,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,NaN,Argentina,-38.416100,-63.616700,0,0,0,0,0,0,...,4,6,8,9,13,18,19,23,27,28
7,NaN,Armenia,40.069100,45.038200,0,0,0,0,0,0,...,0,0,0,1,1,1,3,3,3,4
8,Australian Capital Territory,Australia,-35.473500,149.012400,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
9,New South Wales,Australia,-33.868800,151.209300,0,0,0,0,0,0,...,6,7,7,7,7,8,8,8,8,9


In [264]:
# View raw dataframe of recovered cases
recovered_df

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20
0,NaN,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,...,1,1,2,2,2,2,2,2,5,5
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,2,10,17,17,31,31,33,44,52,67
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,65,24,65,29,29,31,31,37,46,61
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,1,1,1,1,1,1,1,10,10,10
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
5,NaN,Antigua and Barbuda,17.060800,-61.796400,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,NaN,Argentina,-38.416100,-63.616700,0,0,0,0,0,0,...,3,52,52,63,72,72,72,228,240,248
7,NaN,Armenia,40.069100,45.038200,0,0,0,0,0,0,...,2,14,16,18,28,30,30,30,30,31
8,Australian Capital Territory,Australia,-35.473500,149.012400,0,0,0,0,0,0,...,0,0,0,1,1,2,2,2,3,8
9,New South Wales,Australia,-33.868800,151.209300,0,0,0,0,0,0,...,4,4,4,4,4,4,4,4,4,4


---

> ### Step 2 – Transform
>
>
>> #### *Sub-step 2.1 – Clean & Aggregate by Country*
>>
>> Next we cleaned and aggregated the raw data with a defined function in order to: 
>>
>> + fill any NaNs with zero values, 
>> + drop unnecessary columns (*e.g.*, specific latitude and longitude coordinates), 
>> + group and aggregate (i.e., sum) case counts by overall Country/Region in order to eliminate the current breakout by Province/State, 
>> + set the Country/Region as the index, and
>> + set all values as integers, 
>> + sort descending based on the latest calendar date's numbers. 


---

In [265]:
# # Define Function for cleaning the population data
def clean_pop_df(df_pop):
    indexNames = df_pop[df_pop['Type'] != "Country/Area" ].index                 # Getting the index numbers for rows that don't have country datas
    df_pop.drop(indexNames , inplace=True)                              # Dropping Rows which don't have country data
    df_pop = df_pop.drop(columns = ["Variant", "Notes","Parent code", "Index", "Country code", "Type","1950","1951","1952","1953","1954","1955","1956","1957","1958","1959","1960","1961","1962","1963","1964","1965","1966","1967","1968","1969","1970","1971","1972","1973","1974","1975","1976","1977","1978","1979","1980","1981","1982","1983","1984","1985","1986","1987","1988","1989","1990","1991","1992","1993","1994","1995","1996","1997","1998","1999","2000","2001","2002","2003","2004","2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018","2019"])
    df_pop = df_pop.sort_values(by=df_pop.columns[0], ascending=True) # Sorting values
    #df_pop = df_pop.set_index(["Country/Region"])          # Setting Index as region
    df_pop = df_pop.rename(columns={"2020" : "Current Population"})
    df_pop = df_pop.fillna(value=0)                                              # Dropping Null Values
    
    
    return df_pop
df_pop

,Index,Variant,Country/Region,Notes,Country code,Type,Parent code,1950,1951,1952,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,1,Estimates,WORLD,NaN,900,World,0,2.53643e+06,2.58403e+06,2.63086e+06,...,7.04119e+06,7.12583e+06,7.21058e+06,7.29529e+06,7.3798e+06,7.46402e+06,7.54786e+06,7.63109e+06,7.71347e+06,7.7948e+06
1,2,Estimates,UN development groups,a,1803,Label/Separator,900,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,3,Estimates,More developed regions,b,901,Development Group,1803,814819,824004,833720,...,1.23956e+06,1.24411e+06,1.24845e+06,1.25262e+06,1.25662e+06,1.26048e+06,1.26415e+06,1.26756e+06,1.27063e+06,1.2733e+06
3,4,Estimates,Less developed regions,c,902,Development Group,1803,1.72161e+06,1.76003e+06,1.79714e+06,...,5.80164e+06,5.88171e+06,5.96213e+06,6.04268e+06,6.12317e+06,6.20354e+06,6.28371e+06,6.36353e+06,6.44284e+06,6.52149e+06
4,5,Estimates,Least developed countries,d,941,Development Group,902,195428,199180,203015,...,856471,876867,897793,919223,941131,963520,986385,1.00969e+06,1.03339e+06,1.05744e+06
5,6,Estimates,"Less developed regions, excluding least develo...",e,934,Development Group,902,1.52618e+06,1.56085e+06,1.59413e+06,...,4.94517e+06,5.00485e+06,5.06434e+06,5.12345e+06,5.18204e+06,5.24002e+06,5.29733e+06,5.35384e+06,5.40945e+06,5.46406e+06
6,7,Estimates,"Less developed regions, excluding China",NaN,948,Development Group,1803,1.15742e+06,1.17993e+06,1.20396e+06,...,4.39431e+06,4.46655e+06,4.53916e+06,4.612e+06,4.68498e+06,4.75802e+06,4.83109e+06,4.90415e+06,4.9772e+06,5.05021e+06
7,8,Estimates,Land-locked Developing Countries (LLDC),f,1636,Special other,1803,103803,105870,108079,...,430709,441057,451699,462624,473817,485277,496988,508906,520973,533143
8,9,Estimates,Small Island Developing States (SIDS),g,1637,Special other,1803,23771.1,24209.1,24684.6,...,66034.5,66778.7,67490.7,68180.4,68855.5,69515,70157.5,70791.8,71428.8,72076.1
9,10,Estimates,World Bank income groups,NaN,1802,Label/Separator,900,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [266]:
# Cleaning the data
pop_clean = clean_pop_df(df_pop)

In [284]:
# Viewing the Data, population is in thousands
pop_clean

,Country/Region,Current Population
116,afghanistan,38928.341
257,albania,2877.800
82,algeria,43851.043
220,american samoa,55.197
258,andorra,77.265
47,angola,32866.268
149,anguilla,15.002
150,antigua and barbuda,97.928
187,argentina,45195.777
90,armenia,2963.234


In [285]:
#Export to CSV
pop_clean.to_csv("pop_data.csv",index=False)

In [286]:
# Define function for cleaning the data
def clean_df(df, death_confirmed_recovered):
    
    #df = df.fillna(value=0)                                          # Fill NaN with zero values
    df = df.drop(columns=["Lat", "Long"]) # Drop "Lat" and "Long" columns
    df = df.groupby(['Country/Region'], as_index = False).sum() # Create groupby object for sorting by country/region and aggregate by summing   
    df = pd.melt(df, id_vars=["Country/Region"],var_name="Date", value_name= str(death_confirmed_recovered))
    df = df.reset_index(drop=True) # Define new index
    df['Date'] = pd.to_datetime(df['Date']) # convert date column into a date type
    df = df.sort_values(by=["Country/Region", 'Date'], ascending=True) # Sort by highest value of most recently added date column
    return df

In [287]:
# Use defined function (above) to clean each dataframe
confirmed_clean = clean_df(confirmed_df, "Confirmed")
deaths_clean = clean_df(deaths_df, "Deaths")
recovered_clean = clean_df(recovered_df, "Recovered")

In [288]:
# Merge DFs for one transformed table
first_merge = pd.merge(confirmed_clean, deaths_clean, on=['Country/Region', 'Date'])
andrew_table = pd.merge(first_merge, recovered_clean, on=['Country/Region', 'Date'])
andrew_table

,Country/Region,Date,Confirmed,Deaths,Recovered
0,Afghanistan,2020-01-22,0,0,0
1,Afghanistan,2020-01-23,0,0,0
2,Afghanistan,2020-01-24,0,0,0
3,Afghanistan,2020-01-25,0,0,0
4,Afghanistan,2020-01-26,0,0,0
...,...,...,...,...,...
12775,Zimbabwe,2020-03-28,7,1,0
12776,Zimbabwe,2020-03-29,7,1,0
12777,Zimbabwe,2020-03-30,7,1,0
12778,Zimbabwe,2020-03-31,8,1,0


In [290]:
#Normalize Country values
andrew_table["Country/Region"] = andrew_table["Country/Region"].str.lower()
pop_clean["Country/Region"] = pop_clean["Country/Region"].str.lower()


In [292]:
#Rename Unmatched Countries
pop_clean['Country/Region'] = pop_clean['Country/Region'].map(lambda x:x.replace('united states of america','us'))
pop_clean['Country/Region'] = pop_clean['Country/Region'].map(lambda x:x.replace('bolivia','bolivia (plurinational state of)'))
pop_clean['Country/Region'] = pop_clean['Country/Region'].map(lambda x:x.replace('brunei','brunei darussalam'))
pop_clean['Country/Region'] = pop_clean['Country/Region'].map(lambda x:x.replace('burma','myanmar'))
pop_clean['Country/Region'] = pop_clean['Country/Region'].map(lambda x:x.replace("cote d'ivoire","côte d'ivoire"))
pop_clean['Country/Region'] = pop_clean['Country/Region'].map(lambda x:x.replace('iran','iran (islamic republic of)'))
pop_clean['Country/Region'] = pop_clean['Country/Region'].map(lambda x:x.replace('korea, south',"dem. people's republic of korea"))
pop_clean['Country/Region'] = pop_clean['Country/Region'].map(lambda x:x.replace('kosovo',''))
pop_clean['Country/Region'] = pop_clean['Country/Region'].map(lambda x:x.replace('laos',"lao people's democratic republic"))
pop_clean['Country/Region'] = pop_clean['Country/Region'].map(lambda x:x.replace('moldova','republic of moldova	'))
pop_clean['Country/Region'] = pop_clean['Country/Region'].map(lambda x:x.replace('russia','russian federation'))
pop_clean['Country/Region'] = pop_clean['Country/Region'].map(lambda x:x.replace("syria","syrian arab republic"))
pop_clean['Country/Region'] = pop_clean['Country/Region'].map(lambda x:x.replace("taiwan*","china, taiwan province of china	"))
pop_clean['Country/Region'] = pop_clean['Country/Region'].map(lambda x:x.replace("tanzania","united republic of tanzania"))
pop_clean['Country/Region'] = pop_clean['Country/Region'].map(lambda x:x.replace("venezuela","venezuela (bolivarian republic of)"))
pop_clean['Country/Region'] = pop_clean['Country/Region'].map(lambda x:x.replace("vietnam","viet nam"))

In [295]:
#View more rows
pd.set_option('display.max_rows', 1400)

In [296]:
#merge tables on countries
final_merge = pd.merge(andrew_table, pop_clean, how='inner', on='Country/Region')
final_merge

,Country/Region,Date,Confirmed,Deaths,Recovered,Current Population
0,afghanistan,2020-01-22,0,0,0,38928.341
1,afghanistan,2020-01-23,0,0,0,38928.341
2,afghanistan,2020-01-24,0,0,0,38928.341
3,afghanistan,2020-01-25,0,0,0,38928.341
4,afghanistan,2020-01-26,0,0,0,38928.341
...,...,...,...,...,...,...
11355,zimbabwe,2020-03-28,7,1,0,14862.927
11356,zimbabwe,2020-03-29,7,1,0,14862.927
11357,zimbabwe,2020-03-30,7,1,0,14862.927
11358,zimbabwe,2020-03-31,8,1,0,14862.927


In [297]:
# Export cleaned data set into a csv file
final_merge.to_csv("clean_data.csv",index=False)

---

> ### Step 3 – Load
>
>
> Finally we loaded the *transformed* dataframes into respective tables within a relational Postgres SQL database: 
>
> + confirmed cases, 
> + deaths, and 
> + recovered cases.
>

---

Now the latest COVID-19 data has been migrated from the Johns Hopkins University's CSV files into a production SQL database ready for querying, analysis, and visualizations of countries' growth trajectories on comparable timescales.

### Thank you for viewing our ETL project!

#### ~ The CovidCats

---